In [ ]:
import os
import time
import numpy as np
import pandas as pd
import dimod
from neal import SimulatedAnnealingSampler
import pulp


# (그대로 사용) 랜덤 QUBO -> BQM
def make_random_bqm(L, h_range=(-1.0,1.0), J_range=(-1.0,1.0),
                    density=1.0, seed=0):
    rng = np.random.default_rng(seed)
    Q = {}
    for i in range(L):
        Q[(i, i)] = rng.uniform(*h_range)
    for i in range(L):
        for j in range(i+1, L):
            if rng.random() < density:
                Q[(i, j)] = rng.uniform(*J_range)
    return dimod.BinaryQuadraticModel.from_qubo(Q)  # BINARY BQM (QUBO형)

def solve_with_neal(bqm, num_reads=5000, beta_range=(0.1, 4.0), sweeps=1000):
    """SA(Neal)로 최적화: (최저 에너지, 해, 실행 시간[s]) 반환"""
    t0 = time.perf_counter()
    ss = SimulatedAnnealingSampler().sample(
        bqm, num_reads=num_reads, beta_range=beta_range, sweeps=sweeps
    )
    dt = time.perf_counter() - t0
    return ss.first.energy, ss.first.sample, dt  # (최저 에너지, 해, 시간)


# ✅ PuLP를 BQM의 선형/이차 항에서 직접 구성 (QUBO round-trip 불필요)
def solve_with_pulp_from_bqm(bqm, time_limit=None, msg=False):
    """
    PuLP(CBC)로 MILP 변환 후 풀기:
    반환: (obj_val, x_sol, E_check, status_str, 실행 시간[s])
    """
    assert bqm.vartype is dimod.BINARY, "This MILP expects a BINARY BQM (QUBO form)."

    lin = dict(bqm.linear)          # {i: h_i}
    quad = dict(bqm.quadratic)      # {(i,j): J_ij}, i!=j
    offset = float(bqm.offset)      # 상수항

    # 변수 집합
    idxs = sorted(set(lin.keys()) | {i for ij in quad.keys() for i in ij})

    # MILP 모델
    prob = pulp.LpProblem("BQM_to_MILP", pulp.LpMinimize)

    # 이진 변수 x_i (0/1)
    x = {i: pulp.LpVariable(f"x_{i}", lowBound=0, upBound=1, cat=pulp.LpBinary) for i in idxs}

    # 보조 변수 z_ij ~ x_i * x_j (i<j인 쌍만 생성)
    z = {}
    for (i,j), val in quad.items():
        if i == j or abs(val) == 0.0:
            continue
        a, b = (i, j) if i < j else (j, i)
        key = (a, b)
        if key not in z:  # 중복 방지
            z[key] = pulp.LpVariable(f"z_{a}_{b}", lowBound=0, upBound=1, cat=pulp.LpBinary)

    # 목적함수: ∑ h_i x_i + ∑ J_ij z_ij + offset
    obj_terms = []
    for i, h in lin.items():
        obj_terms.append(h * x[i])
    for (i,j), J in quad.items():
        if i == j or abs(J) == 0.0:
            continue
        a, b = (i, j) if i < j else (j, i)
        obj_terms.append(J * z[(a, b)])
    prob += pulp.lpSum(obj_terms) + offset

    # 선형화 제약: z_ij = x_i * x_j  (binary McCormick)
    for (i,j), zij in z.items():
        prob += zij <= x[i]
        prob += zij <= x[j]
        prob += zij >= x[i] + x[j] - 1

    # 풀기 (CBC)
    solver = pulp.PULP_CBC_CMD(msg=msg, timeLimit=time_limit) if time_limit else pulp.PULP_CBC_CMD(msg=msg)

    t0 = time.perf_counter()
    prob.solve(solver)
    dt = time.perf_counter() - t0

    status_str = pulp.LpStatus.get(prob.status, str(prob.status))

    # 해 추출 (해가 없을 수도 있으므로 방어)
    if status_str not in ("Optimal", "Not Solved", "Infeasible", "Unbounded", "Undefined", "Integer Feasible"):
        status_str = f"Status({prob.status})"

    x_sol = {i: int(round(pulp.value(var))) for i, var in x.items()} if status_str not in ("Infeasible", "Unbounded", "Undefined") else {}
    obj_val = float(pulp.value(prob.objective)) if x_sol else np.nan

    # 동일한 BQM으로 재평가하여 일관성 확인
    E_check = bqm.energy(x_sol) if x_sol else np.nan

    return obj_val, x_sol, E_check, status_str, dt

def sanity_check_small_L(L=10, density=1.0, seed=0):
    bqm = make_random_bqm(L, density=density, seed=seed)

    # Exact (작은 L에서만)
    exact_t0 = time.perf_counter()
    exact = dimod.ExactSolver().sample(bqm).first
    exact_dt = time.perf_counter() - exact_t0
    E_exact = exact.energy

    # PuLP
    E_pulp, x_pulp, E_check, status, pulp_dt = solve_with_pulp_from_bqm(bqm)
    # Neal (SA)
    E_neal, x_neal, neal_dt = solve_with_neal(bqm)

    print(f"[Exact] E={E_exact:.6f} | time={exact_dt:.4f}s")
    print(f"[PuLP ] status={status:>9s} | E_pulp={E_pulp:.6f}, E_check={E_check:.6f}, gap_to_exact={E_pulp - E_exact:.6e} | time={pulp_dt:.4f}s")
    print(f"[Neal ] E={E_neal:.6f} (heuristic) | time={neal_dt:.4f}s")

if __name__ == "__main__":
    # 1) 작은 L에서 검증 (옵션)
    sanity_check_small_L(L=12, density=1.0, seed=42)

    # 2) 큰 L 루프
    L_values = list(range(5, 101, 5))
    density = 1.0
    # SA(Neal)
    neal_reads = 2000
    neal_sweeps = 1000

    # PuLP(MILP)
    pulp_time_limit = 3600  # 예: 1시간 제한

    records = []

    for L in L_values:
        seed = L  # 재현성: 사이즈별로 seed 고정
        bqm = make_random_bqm(L, density=density, seed=seed)

        # SA (Neal)
        En, xN, tN = solve_with_neal(bqm, num_reads=neal_reads, sweeps=neal_sweeps)

        # PuLP (CBC)
        Ep, xP, Ep_check, status, tP = solve_with_pulp_from_bqm(bqm, time_limit=pulp_time_limit, msg=False)

        print(
            f"L={L:2d} | Neal: E={En: .6f}, time={tN: .3f}s | "
            f"PuLP [{status}]: E={Ep: .6f}, E_check={Ep_check: .6f}, time={tP: .3f}s"
        )

        records.append({
            "L": L,
            "density": density,
            "seed": seed,
            # Neal(SA)
            "sa_energy": En,
            "sa_time_sec": tN,
            "sa_num_reads": neal_reads,
            "sa_sweeps": neal_sweeps,
            # PuLP(MILP)
            "pulp_energy": Ep,
            "pulp_time_sec": tP,
            "pulp_status": status,
            "pulp_E_check": Ep_check,
            # 차이(참고용)
            "gap_pulp_minus_sa": (Ep - En) if (np.isfinite(Ep) and np.isfinite(En)) else np.nan
        })

    df = pd.DataFrame.from_records(records)
    out_csv = "results_qubo_sa_pulp.csv"
    df.to_csv(out_csv, index=False)
    print(f"\nSaved: {os.path.abspath(out_csv)}")
    print(df.head(10).to_string(index=False))


[Exact] E=-7.419472 | time=0.0036s
[PuLP ] status=  Optimal | E_pulp=-7.419472, E_check=-7.419472, gap_to_exact=0.000000e+00 | time=0.0315s
[Neal ] E=-7.419472 (heuristic) | time=0.7888s
L= 5 | Neal: E=-0.962173, time= 0.103s | PuLP [Optimal]: E=-0.962173, E_check=-0.962173, time= 0.015s
L=10 | Neal: E=-4.144075, time= 0.279s | PuLP [Optimal]: E=-4.144075, E_check=-4.144075, time= 0.026s
L=15 | Neal: E=-4.693355, time= 0.438s | PuLP [Optimal]: E=-4.693355, E_check=-4.693355, time= 0.432s
L=20 | Neal: E=-21.918864, time= 0.583s | PuLP [Optimal]: E=-21.918864, E_check=-21.918864, time= 0.876s
L=25 | Neal: E=-20.223399, time= 0.756s | PuLP [Optimal]: E=-20.223399, E_check=-20.223399, time= 2.181s
L=30 | Neal: E=-27.650854, time= 0.927s | PuLP [Optimal]: E=-27.650854, E_check=-27.650854, time= 7.327s
L=35 | Neal: E=-44.637908, time= 1.162s | PuLP [Optimal]: E=-44.637908, E_check=-44.637908, time= 9.539s


In [1]:
import os, time
import numpy as np
import pandas as pd
import dimod
from neal import SimulatedAnnealingSampler

# ---------- QUBO 생성 ----------
def make_random_bqm(L, h_range=(-1.0,1.0), J_range=(-1.0,1.0), density=1.0, seed=0):
    rng = np.random.default_rng(seed)
    Q = {}
    for i in range(L):
        Q[(i, i)] = rng.uniform(*h_range)
    for i in range(L):
        for j in range(i+1, L):
            if rng.random() < density:
                Q[(i, j)] = rng.uniform(*J_range)
    return dimod.BinaryQuadraticModel.from_qubo(Q)

# ---------- SA (neal) ----------
def solve_with_sa(bqm, num_reads=2000, sweeps=1000, beta_range=(0.1,4.0), seed=None):
    t0 = time.perf_counter()
    ss = SimulatedAnnealingSampler().sample(
        bqm, num_reads=num_reads, sweeps=sweeps, beta_range=beta_range, seed=seed
    )
    dt = time.perf_counter() - t0
    energies = ss.record.energy.astype(float)
    best_e = float(energies.min())
    best_sample = ss.record.sample[np.argmin(energies)]
    varlist = list(bqm.variables)
    best_x = {varlist[i]: int(best_sample[i]) for i in range(len(varlist))}
    return best_e, float(energies.mean()), float(energies.std()), dt, best_x

# ---------- Gurobi(MIQP) ----------
def solve_with_gurobi_from_bqm(bqm,
                               time_limit=None, mip_gap=None,
                               threads=None, seed=None,
                               warm_start=None, log_to_console=False):
    """
    QUBO(BINARY BQM)를 Gurobi MIQP로 직접 풀기.
    반환: (obj_val, x_sol(dict), E_check(float), status_str, runtime_sec)
    """
    import gurobipy as gp
    from gurobipy import GRB

    if bqm.vartype is not dimod.BINARY:
        raise ValueError("This solver expects a BINARY BQM (QUBO).")

    t0 = time.perf_counter()
    m = gp.Model("QUBO_MIQP")
    if not log_to_console:
        m.Params.OutputFlag = 0
    if time_limit is not None:
        m.Params.TimeLimit = float(time_limit)
    if mip_gap is not None:
        m.Params.MIPGap = float(mip_gap)
    if threads is not None:
        m.Params.Threads = int(threads)
    if seed is not None:
        m.Params.Seed = int(seed)

    # 중요: 비볼록 QP 허용 (일반 QUBO는 비정합일 수 있음)
    m.Params.NonConvex = 2

    # 변수
    vars_map = {v: m.addVar(vtype=GRB.BINARY, name=f"x_{v}") for v in bqm.variables}
    m.update()

    # 목적함수: sum_i h_i x_i + sum_{i<j} J_ij x_i x_j + offset
    obj_lin = gp.LinExpr()
    for i, h in bqm.linear.items():
        obj_lin += float(h) * vars_map[i]

    obj_quad = gp.QuadExpr()
    for (i, j), J in bqm.quadratic.items():
        if i == j:
            continue
        obj_quad += float(J) * vars_map[i] * vars_map[j]

    m.setObjective(obj_lin + obj_quad + float(bqm.offset), GRB.MINIMIZE)

    # Warm start (선택): 초기해 제공
    if isinstance(warm_start, dict):
        for i, var in vars_map.items():
            val = warm_start.get(i, 0)
            var.Start = int(val)

    m.optimize()
    runtime = time.perf_counter() - t0

    # 상태
    status_map = {
        GRB.OPTIMAL: "OPTIMAL",
        GRB.TIME_LIMIT: "TIME_LIMIT",
        GRB.INTERRUPTED: "INTERRUPTED",
        GRB.INFEASIBLE: "INFEASIBLE",
        GRB.INF_OR_UNBD: "INF_OR_UNBD",
        GRB.UNBOUNDED: "UNBOUNDED",
    }
    status_str = status_map.get(m.Status, f"STATUS_{m.Status}")

    x_sol = {}
    if m.SolCount > 0:
        x_sol = {i: int(round(var.X)) for i, var in vars_map.items()}
        obj_val = float(m.ObjVal)
        E_check = float(bqm.energy(x_sol))
    else:
        obj_val = float("nan")
        E_check = float("nan")

    return obj_val, x_sol, E_check, status_str, runtime

# ---------- 메인(예시): SA vs Gurobi ----------
if __name__ == "__main__":
    L_values = list(range(5, 101, 5))
    density   = 1.0
    seed_base = 0#123

    # SA 설정
    sa_reads, sa_sweeps = 2000, 1000

    # Gurobi 설정
    time_limit = 3600       # 초 (필요시 늘려도 됨)
    mip_gap    = 1e-4
    threads    = None      # CPU 코어 수에 맞춰 조정 가능
    use_warm   = True      # SA warm start 사용 여부

    rows = []
    for L in L_values:
        seed = seed_base + L
        bqm  = make_random_bqm(L, density=density, seed=seed)

        # 1) SA (비교 + warm start 생성)
        sa_best, sa_mean, sa_std, sa_time, sa_warm = solve_with_sa(
            bqm, num_reads=sa_reads, sweeps=sa_sweeps, seed=seed
        )

        # 2) Gurobi (MIQP) — SA warm start 사용(옵션)
        warm = sa_warm if use_warm else None
        grb_obj, grb_x, grb_E_check, grb_status, grb_time = solve_with_gurobi_from_bqm(
            bqm,
            time_limit=time_limit, mip_gap=mip_gap, threads=threads, seed=seed,
            warm_start=warm, log_to_console=False
        )

        print(
            f"L={L:2d} | "
            f"SA   best={sa_best: .6f}, mean={sa_mean: .6f}±{sa_std: .6f}, t={sa_time: .2f}s  ||  "
            f"Gurobi [{grb_status}] obj={grb_obj: .6f}, E_check={grb_E_check: .6f}, t={grb_time: .2f}s"
        )

        rows.append({
            "L": L, "seed": seed, "density": density,
            # SA
            "sa_best": sa_best, "sa_mean": sa_mean, "sa_std": sa_std, "sa_time_sec": sa_time,
            "sa_reads": sa_reads, "sa_sweeps": sa_sweeps,
            # Gurobi
            "grb_status": grb_status, "grb_obj": grb_obj, "grb_E_check": grb_E_check, "grb_time_sec": grb_time,
            "grb_time_limit": time_limit, "grb_mip_gap": mip_gap, "grb_threads": threads,
            "grb_warmstart": use_warm,
            # 비교
            "gap_grb_minus_sa": (grb_obj - sa_best) if np.isfinite(grb_obj) and np.isfinite(sa_best) else np.nan,
            "speed_ratio_grb_over_sa": (grb_time / sa_time) if sa_time > 0 else np.nan,
        })

    df = pd.DataFrame(rows)
    out_csv = "results_qubo_sa_vs_gurobi_miqp.csv"
    df.to_csv(out_csv, index=False)
    print(f"\nSaved: {os.path.abspath(out_csv)}")
    print(df.head(10).to_string(index=False))


Set parameter Username

--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only - expires 2025-09-25
L= 5 | SA   best=-0.962173, mean=-0.709398± 0.334410, t= 0.10s  ||  Gurobi [OPTIMAL] obj=-0.962173, E_check=-0.962173, t= 0.00s
L=10 | SA   best=-4.144075, mean=-3.803262± 0.401422, t= 0.29s  ||  Gurobi [OPTIMAL] obj=-4.144075, E_check=-4.144075, t= 0.00s
L=15 | SA   best=-4.693355, mean=-4.211982± 0.456946, t= 0.44s  ||  Gurobi [OPTIMAL] obj=-4.693355, E_check=-4.693355, t= 0.01s
L=20 | SA   best=-21.918864, mean=-21.698732± 0.349347, t= 0.58s  ||  Gurobi [OPTIMAL] obj=-21.918864, E_check=-21.918864, t= 0.02s
L=25 | SA   best=-20.223399, mean=-19.944004± 0.371893, t= 0.76s  ||  Gurobi [OPTIMAL] obj=-20.223399, E_check=-20.223399, t= 0.06s
L=30 | SA   best=-27.650854, mean=-27.280257± 0.458544, t= 0.95s  ||  Gurobi [OPTIMAL] obj=-27.650854, E_check=-27.650854, t= 0.12s
L=35 | SA   best=-44.637908, mean=-44